# JSON Schema

A JSON schema declares the expected format of a JSON document.

Regular JSON is human-readable, but it doesn't include any context or metadata. JSON Schema enables information about the meanings of the keys and the possible values they can take.

It is a popular method for documenting and validating JSON files.

For more check the [documentation](https://json-schema.org/docs) of JSON schema.

A JSON Schema is a standard used to define and validate the structure and data types of a JSON document (the instance). It is itself a JSON document where the structure and constraints for each **property** are described using predefined **schema keywords** (e.g., type, properties).

For examples chema:

In [20]:
echo '{"type": "number"}' > /tmp/schema.json

Specifies the the root level of the JSON should be a number, e.g. JSON by itself a number.

In [21]:
echo 10 > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

ok -- validation done


The following cell specifies that the root object is a mapping. The available attributes of the root mapping are specified in the `properties` of the JSON schema.

In [23]:
cat << EOF > /tmp/schema.json
{
    "type": "object",
    "properties":
    {
        "wow": {
            "type": "string"
        }
    }
}
EOF
echo '{"wow": "value"}' > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json

ok -- validation done


## Type

The `type` keyword is used to specify the type of the attribute.

Check more in [Type-specified Keywords](https://json-schema.org/understanding-json-schema/reference/type).

---

The following cell indicates that the root element of the JSON should be a string.

In [36]:
echo '{"type": "string"}' > /tmp/schema.json

Specifying a string works fine.

In [38]:
echo '"some string"' > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

ok -- validation done


However, any other object, such as number, will result in the corresponding error.

In [40]:
echo 10 > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

Schema validation errors were encountered.
  /tmp/object.json::$: 10 is not of type 'string'


## Arrays

An array is used to list elements. Find out more about arrays in JSON schemas on the [Array](https://json-schema.org/understanding-json-schema/reference/array) reference page.

Use the following schema keywords to define what the items array can contain:

- [`items`](https://json-schema.org/understanding-json-schema/reference/array#items): This sepecifies the schema to which each item in the list must correspond. Such approach is called **List validation**.
- `prefixItems`: peforms the [tuple validation](https://json-schema.org/understanding-json-schema/reference/array#tupleValidation) where the order of items is important and each item has a different schema.
- [`contains`](https://json-schema.org/understanding-json-schema/reference/array#contains): specifies the rule to which at least one element of the array have to correspond.

**Note**: You can combine items with different specifications for the same array.

---

The following cell defines the schema for array for which:

- `items`: sets up that array can contains only numbers 1, 2, 3.
- `contains`: defines that it have to contain at least one entrance of 2.

In [3]:
cat << EOF > /tmp/schema.json
{
    "type": "array",
    "items": {
        "enum": [1, 2, 3]
    },
    "contains": {
        "const": 2
    }
}
EOF

The array `[3, 3, 2]` is fine because it contains elements defined in the `contains` and does not contain elements that are not mentioned in the `items`.

In [8]:
echo "[3, 3, 2]" > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

ok -- validation done


The case where the item `2` is not present is validated in the following cell.

In [10]:
echo "[1, 1, 1, 3]" > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

Schema validation errors were encountered.
  /tmp/object.json::$: [1, 1, 1, 3] does not contain items matching the given schema


The case where the array under consideration contains a value that is not mentioned in the `items` is represented in the following cell.

In [9]:
echo "[1, 4, 2]" > /tmp/object.json
check-jsonschema --schemafile /tmp/schema.json /tmp/object.json
true

Schema validation errors were encountered.
  /tmp/object.json::$[1]: 4 is not one of [1, 2, 3]


## Composition

You can combine different schemas to create a complex validation algorithms. There are two concepts:

- **Modular**: allows to reuse defined schemas in other schemas.
- **Boolean**: you can specify a schema that validates the input against several schemas joined by logical operations.

Check more in [Schema Composition](https://json-schema.org/understanding-json-schema/reference/composition) page.

### Modularity

The JSON Schema syntax enables reusable JSON structures to be specified and referenced by other structures. These structures can be defined in external JSON Schema files or within the same file.

The following keys are used to define and reference other schemas:

- `$id`: Defines the URI used to identify this root schema.
- `$anchor`: Defines a specific subschema location that can be referenced by a fragment identifier.
- `$defs`: A mapping key where other subschemas are defined so they can be referenced.
- `$ref`: Indicates that the current element should be validated against the schema at the specific URI provided in the value.

Learn more on the [Modular JSON Schema Validation](https://json-schema.org/understanding-json-schema/structuring) page.

---

The following cell defines the JSON schema that defines `"PC"` and `"Server"` attributes both of which require one of `"windows", "linux", "freeBSD"`.

In [15]:
cat << EOF > /tmp/schema.json
{
    "type": "object",
    "properties": {
        "PC": {"$ref": "#/$defs/system"},
        "Server": {"$ref": "#/$defs/system"}
    },
    "$defs": {
        "system": {"enum": ["windows", "linux", "freeBSD"]}
    }
}
EOF

**Note:** for some reason, `check-jsonschema` doesn't work with references. Try using the [JSON schema validator](https://www.jsonschemavalidator.net/).